# 1. Set up parameters for ASL

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
from obspy import read_inventory
from importlib import reload
from flovopy.asl.wrappers import run_single_event, find_event_files, run_all_events
from flovopy.core.mvo import dome_location, REGION_DEFAULT
from flovopy.processing.sam import VSAM, DSAM 
from flovopy.asl.config import ASLConfig
# -------------------------- Config --------------------------
# directories
HOME = Path.home()
PROJECTDIR      = HOME / "Dropbox" / "BRIEFCASE" / "SSADenver"
LOCALPROJECTDIR = HOME / "work" / "PROJECTS" / "SSADenver_local"
OUTPUT_DIR      = LOCALPROJECTDIR / "ASL_RESULTS"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
INPUT_DIR       = PROJECTDIR / "ASL_inputs" / "biggest_pdc_events"
GLOBAL_CACHE    = PROJECTDIR / "asl_global_cache"
METADATA_DIR    = PROJECTDIR / "metadata" 
STATION_CORRECTIONS_DIR = PROJECTDIR / "station_correction_analysis"

# master files
INVENTORY_XML   = METADATA_DIR / "MV_Seismic_and_GPS_stations.xml"
DEM_DEFAULT     = METADATA_DIR / "MONTSERRAT_DEM_WGS84_MASTER.tif"
GRIDFILE_DEFAULT= METADATA_DIR / "MASTER_GRID_MONTSERRAT.pkl"

# parameters for envelopes and cross-correlation
SMOOTH_SECONDS  = 1.0
MAX_LAG_SECONDS = 8.0
MIN_XCORR       = 0.5

# other parameters
DIST_MODE = "3d" # or 2d. will essentially squash Montserrat topography and stations onto a sea-level plane, ignored elevation data, e.g. for computing distances

# Inventory of Montserrat stations
from obspy import read_inventory
INV     = read_inventory(INVENTORY_XML)
print(f"[INV] Networks: {len(INV)}  Stations: {sum(len(n) for n in INV)}  Channels: {sum(len(sta) for net in INV for sta in net)}")

# Montserrat station corrections estimated from regionals
station_corrections_csv = STATION_CORRECTIONS_DIR / "station_gains_intervals.csv"
annual_station_corrections_csv = STATION_CORRECTIONS_DIR / "station_gains_intervals_by_year.csv"
station_corrections_df = pd.read_csv(station_corrections_csv)
annual_station_corrections_df = pd.read_csv(annual_station_corrections_csv)

# Montserrat pre-defined Grid (from 02 tutorial)
from flovopy.asl.grid import Grid
gridobj = Grid.load(GRIDFILE_DEFAULT)
print(gridobj)


# Montserrat constants
from flovopy.core.mvo import dome_location, REGION_DEFAULT
print("Dome (assumed source) =", dome_location)

# events and wrappers
event_files = list(find_event_files(INPUT_DIR))
eventcsvfile = Path(OUTPUT_DIR) / "mseed_files.csv"
if not eventcsvfile.is_file():
    rows = [{"num": num, "f": str(f)} for num, f in enumerate(event_files)]
    df = pd.DataFrame(rows)
    df.to_csv(eventcsvfile, index=False)
best_file_nums  = [35, 36, 40, 52, 82, 83, 84, 116, 310, 338]
best_event_files = [event_files[i] for i in best_file_nums]
print(f'Best miniseed files are: {best_event_files}')
REFINE_SECTOR = False   # enable triangular dome-to-sea refinement

# Parameters to pass for making pygmt topo maps
topo_kw = {
    "inv": INV,
    "add_labels": True,
    "cmap": "gray",
    "region": REGION_DEFAULT,
    "dem_tif": DEM_DEFAULT,  # basemap shading from your GeoTIFF - but does not actually seem to use this unless topo_color=True and cmap=None
    "frame": True,
    "dome_location": dome_location,
}

# Build a baseline configuration
This is inherited by various downstream functions
This describes the physical parameters, the station metadata, the grid, the misfit algorithm, etc.

In [ ]:
DEBUG=False
baseline_cfg = ASLConfig(
    inventory=INV,
    output_base=OUTPUT_DIR,
    gridobj=gridobj,
    global_cache=GLOBAL_CACHE,
    station_correction_dataframe=station_corrections_df,
    wave_kind="surface",
    speed=1.5,
    Q=23, 
    peakf=2.0,
    dist_mode="3d", 
    misfit_engine="r2",
    window_seconds=5.0,
    min_stations=5,
    sam_class=VSAM, 
    sam_metric="mean",
    debug=DEBUG,
)
baseline_cfg.build()


# Run events (one event=one miniseed file) with this baseline configuration

In [ ]:
summaries = []
REFINE_SECTOR=False
for i, ev in zip(best_file_nums, best_event_files):
    print(f"[{i}/{len(event_files)}] {ev}")
    result = run_single_event(
        mseed_file=str(ev),
        cfg=baseline_cfg,
        refine_sector=REFINE_SECTOR,
        station_gains_df=None,
        switch_event_ctag = True,
        topo_kw=topo_kw,
        mseed_units='m/s', # default units for miniseed files being used - probably "Counts" or "m/s"        
        reduce_time=True,
        debug=DEBUG,
    )
    summaries.append(result)

# Summarize
df = pd.DataFrame(summaries)
display(df)

summary_csv = Path(OUTPUT_DIR) / f"{baseline_cfg.tag()}__summary.csv"
df.to_csv(summary_csv, index=False)
print(f"Summary saved to: {summary_csv}")

if not df.empty:
    n_ok = int((~df.get("error").notna()).sum()) if "error" in df.columns else len(df)
    print(f"Success: {n_ok}/{len(df)}")

# Easily create new configurations that change 1 or 2 parameters
Here we create 18 new configurations, each is an entry in the changes dictionary.

In [ ]:
from flovopy.asl.compare_runs import tweak_config, compare_runs
landgridobj = Grid.load(GLOBAL_CACHE / "land" / "Grid_9c2fd59b.pkl")

variants = tweak_config(
    baseline_cfg,
    changes=[
        {"Q": 10},                                      # decrease Q from 23 to 10
        {"Q": 100},                                     # increase Q from 23 to 100
        {"speed": 0.5},                                 # decrease wave speed from 1.5 km/s to 0.5 km/s
        {"speed": 2.5},                                 # increase wave speed from 1.5 km/s to 2.5 km/s
        {"peakf": 8.0},                                 # increae peakf from 2.0 Hz to 8.0 Hz
        {"dist_mode": "2d"},                            # change from 3D to 2D: ignore terrain & ignore station elevations
        {"station_correction_dataframe": None},         # turn off station corrections
        {"gridobj":landgridobj},                        # try a grid that allows whole Southern end of island, not just dome & ravines
        {"misfit_engine": "l2"},                        # change the misfit function from r2 to l2
        {"misfit_engine": "lin"},                       # change the misfit function from r2 to lin
        {"window_seconds": 1.0},                        # decrease the moving window length from 5-s to 1-s
        {"sam_class": DSAM},                            # switch from VELOCITY Seismic Amplitude Measurement (VSAM) to DISPLACEMENT Seismic Amplitude Measurement (DSAM)
        {"sam_metric": "median"},                       # switch from MEAN of each 5-s moving time window, to MEDIAN
        {"sam_metric": "rms"},                          # switch from MEAN of each 5-s moving time window, to RMS
        {"sam_metric": "max"},                          # switch from MEAN of each 5-s moving time window, to MAX
        {"sam_metric": "LP"},                           # switch from MEAN in 0.5-18.0 Hz band to mean in LP band (0.5-4.0 Hz)
        {"sam_metric": "VT"},                           # switch from MEAN in 0.5-18.0 Hz band to mean in VT band (4.0-18.0 Hz)
        {"wave_kind": "body", "speed": 2.5},            # change multiple params: surface->body waves, wave speed 1.5->3.0 km/s - THIS IS A REFERENCE TO COMPARE SURFACE WAVES AND BODY WAVES
    ],
)
print(variants)

# Run pairs: variant vs baseline

In [ ]:
scored, summary, win_counts = compare_runs(
    baseline_cfg,
    events=best_event_files,
    variants=variants,
    run_single_event=run_single_event,
    refine_sector=False,
    topo_kw=topo_kw,
    run_if_missing_baseline=True,
    run_if_missing_variants=True,
    # the following numbers should add up to 1.0. if truly just want to see difference RELATIVE to baseline, set w_sep=1.0, and others to 0.0. for ABSOLUTE quality check, set w_sep to 0.0
    w_sep=0.5, # set this high to penalize large location difference from the baseline config
    w_misfit=0.2, # set this high to punish high misfits
    w_azgap=0.1, # punish larger azimuthal gaps
    w_conn=0.1, # reward more connectedness
    w_rough=0.1, # reward less roughness / more straightness
)

# 4) Inspect/save
if scored is not None:
    display(summary)
    display(win_counts)
    summary.to_csv(OUTPUT_DIR / "pairwise_summary_surface.csv", index=False)

## baseline-free absolute scoring


In [ ]:
from flovopy.asl.compare_runs import build_intrinsic_table, add_baseline_free_scores, summarize_absolute_runs, per_event_winner_abs, crawl_intrinsic_runs

'''
# Option A: ensure CSVs by (re)running baseline + variants you care about
abs_tbl = build_intrinsic_table(
    baseline_cfg,
    events=best_event_files,
    variants=variants,
    run_single_event=run_single_event,
    refine_sector=False, topo_kw=topo_kw,
    run_if_missing_baseline=False,     # set True if you want it to run
    run_if_missing_variants=False,     # set True to auto-run variants
)
'''

# Option B: OR just crawl everything that already exists under OUTPUT_DIR
abs_tbl = crawl_intrinsic_runs(OUTPUT_DIR)
weights = {
    "mean_misfit":     1.0,   # lower better
    "mean_azgap":      0.1,   # lower better
    "roughness_ratio": 0.1,   # lower better
    "connectedness":  -0.3,   # higher better (negative weight)
    "valid_frac":     -0.2,   # higher better (negative weight)
}
abs_scored = add_baseline_free_scores(abs_tbl, weights=weights)

abs_summary = summarize_absolute_runs(abs_scored)
winners_abs, win_counts_abs = per_event_winner_abs(abs_scored)

display(abs_summary)
display(win_counts_abs)

abs_summary.to_csv(OUTPUT_DIR / "absolute_summary.csv", index=False)

# Now let's try to run a suite of body wave configurations


In [ ]:
# Create new baseline configuration - this time for body waves
baseline_cfg = ASLConfig(
    inventory=INV,
    output_base=OUTPUT_DIR,
    gridobj=gridobj,
    global_cache=GLOBAL_CACHE,
    station_correction_dataframe=station_corrections_df,
    wave_kind="body",
    speed=2.5,
    Q=23, 
    peakf=2.0,
    dist_mode="3d", 
    misfit_engine="r2",
    window_seconds=5.0,
    min_stations=5,
    sam_class=VSAM, 
    sam_metric="mean",
    debug=DEBUG,
)
baseline_cfg.build()

# Easily create new configurations - 18 new ones
variants = tweak_config(
    baseline_cfg,
    changes=[
        {"Q": 10},                                      # decrease Q from 23 to 10
        {"Q": 100},                                     # increase Q from 23 to 100
        {"speed": 1.5},                                 # decrease wave speed from 2.5 km/s to 1.5 km/s
        {"speed": 4.0},                                 # increase wave speed from 2.5 km/s to 4.0 km/s
        {"peakf": 8.0},                                 # increae peakf from 2.0 Hz to 8.0 Hz
        {"dist_mode": "2d"},                            # change from 3D to 2D: ignore terrain & ignore station elevations
        {"station_correction_dataframe": None},         # turn off station corrections
        {"gridobj":landgridobj},                        # try a grid that allows whole Southern end of island, not just dome & ravines
        {"misfit_engine": "l2"},                        # change the misfit function from r2 to l2
        {"misfit_engine": "lin"},                       # change the misfit function from r2 to lin
        {"window_seconds": 1.0},                        # decrease the moving window length from 5-s to 1-s
        {"sam_class": DSAM},                            # switch from VELOCITY Seismic Amplitude Measurement (VSAM) to DISPLACEMENT Seismic Amplitude Measurement (DSAM)
        {"sam_metric": "median"},                       # switch from MEAN of each 5-s moving time window, to MEDIAN
        {"sam_metric": "rms"},                          # switch from MEAN of each 5-s moving time window, to RMS
        {"sam_metric": "max"},                          # switch from MEAN of each 5-s moving time window, to MAX
        {"sam_metric": "LP"},                           # switch from MEAN in 0.5-18.0 Hz band to mean in LP band (0.5-4.0 Hz)
        {"sam_metric": "VT"},                           # switch from MEAN in 0.5-18.0 Hz band to mean in VT band (4.0-18.0 Hz)
        {"wave_kind": "surface", "speed": 1.0},         # change multiple params: surface->body waves, wave speed 1.5->3.0 km/s - THIS IS A REFERENCE TO COMPARE SURFACE WAVES AND BODY WAVES
    ],
)

# Run pairs - variant versus baseline
scored, summary, win_counts = compare_runs(
    baseline_cfg,
    events=best_event_files,
    variants=variants,
    run_single_event=run_single_event,
    refine_sector=False,
    topo_kw=topo_kw,
    run_if_missing_baseline=True,
    run_if_missing_variants=True,
    # the following numbers should add up to 1.0. if truly just want to see difference RELATIVE to baseline, set w_sep=1.0, and others to 0.0. for ABSOLUTE quality check, set w_sep to 0.0
    w_sep=0.5, # set this high to penalize large location difference from the baseline config
    w_misfit=0.2, # set this high to punish high misfits
    w_azgap=0.1, # punish larger azimuthal gaps
    w_conn=0.1, # reward more connectedness
    w_rough=0.1, # reward less roughness / more straightness
)

if scored is not None:
    display(summary)
    display(win_counts)
    summary.to_csv(OUTPUT_DIR / "pairwise_summary_surface.csv", index=False)


# baseline-free absolute scoring
abs_tbl = crawl_intrinsic_runs(OUTPUT_DIR)
weights = {
    "mean_misfit":     1.0,   # lower better
    "mean_azgap":      0.1,   # lower better
    "roughness_ratio": 0.1,   # lower better
    "connectedness":  -0.3,   # higher better (negative weight)
    "valid_frac":     -0.2,   # higher better (negative weight)
}
abs_scored = add_baseline_free_scores(abs_tbl, weights=weights)

abs_summary = summarize_absolute_runs(abs_scored)
winners_abs, win_counts_abs = per_event_winner_abs(abs_scored)

display(abs_summary)
display(win_counts_abs)

abs_summary.to_csv(OUTPUT_DIR / "absolute_summary.csv", index=False)

# Run this

In [ ]:
from fred import bloggs
from flovopy.asl.compare_runs import (
    tweak_config, compare_runs,
    load_all_event_comparisons, summarize_variants, per_event_winner
)

REFINE_SECTOR = False
landgridobj = Grid.load(GLOBAL_CACHE / "land" / "Grid_9c2fd59b.pkl")

# 2) Variants
variants = tweak_config(
    baseline_cfg,
    changes=[
        {"Q": 100},
        {"speed": 3.0},
        {"dist_mode": "2d"},
        {"station_correction_dataframe": None},        # turn off stacorr
        {"wave_kind": "body", "speed": 3.2, "Q": 80},  # change multiple params
    ],
)
'''
# generate a sweep
variants = tweak_config(
    baseline_cfg,
    axes={
        "speed": [1.5, 3.0],
        "Q": [23, 100],
    },
    changes=[{"dist_mode": "2d"}],  # also try all of the above in 2D
)

# support previous version
variants = tweak_config(
    baseline_cfg,
    landgridobj=landgridobj,
    annual_station_corrections_df=annual_station_corrections_df,
)
'''

# 3) Run
scored, summary, win_counts = compare_runs(
    baseline_cfg,
    events=best_event_files,
    variants=variants,
    run_single_event=run_single_event,
    refine_sector=False,
    topo_kw=topo_kw,
    run_if_missing_baseline=True,
    run_if_missing_variants=True,
    w_sep=1.0, w_misfit=0.5, w_azgap=0.1,
)

# 4) Inspect/save
if scored is not None:
    display(summary)
    display(win_counts)
    summary.to_csv(OUTPUT_DIR / "pairwise_summary_surface.csv", index=False)

In [ ]:

# ✅ use the new module:
from flovopy.asl.compare_runs2 import (
    cfg_variants_from,
    compare_runs,            # orchestrator
    load_all_event_comparisons,  # optional if you want to re-load later
    add_composite_score,         # optional
    summarize_variants,          # optional
    per_event_winner,            # optional
)

# --- Build baseline & variants ---
landgridobj = Grid.load(GLOBAL_CACHE / "land" / "Grid_9c2fd59b.pkl")
baseline_cfg = ASLConfig(
    inventory=INV,
    output_base=OUTPUT_DIR,
    gridobj=gridobj,
    global_cache=GLOBAL_CACHE,
    station_correction_dataframe=station_corrections_df,
    wave_kind="surface",
    speed=1.5,
    Q=23,
    peakf=2.0,
    dist_mode="3d",
    misfit_engine="r2",
    window_seconds=5.0,
    min_stations=5,
    sam_class=VSAM,
    sam_metric="mean",
    debug=False,
).build()

variants = cfg_variants_from(
    baseline_cfg,
    landgridobj=landgridobj,
    annual_station_corrections_df=annual_station_corrections_df,
)

# --- Run comparisons (auto-run baseline & variants if missing) ---
scored, summary, win_counts = compare_runs(
    baseline_cfg,
    events=best_event_files,            # iterable of mseed paths
    variants=variants,
    run_single_event=run_single_event,  # your existing function
    refine_sector=False,
    topo_kw=topo_kw,
    run_if_missing_baseline=True,
    run_if_missing_variants=True,       # flip to False if you want variants manual
    w_sep=1.0, w_misfit=0.5, w_azgap=0.1,
)

# --- Show results ---
if scored is not None:
    display(summary)
    display(win_counts)
summary.to_csv(OUTPUT_DIR / "pairwise_summary_surface.csv", index=False)

In [ ]:
from flovopy.asl.compare_runs import compare_runs, build_intrinsic_table, add_baseline_free_scores, summarize_absolute_runs, per_event_winner_abs

# 1) (optional) still run pairwise comparisons
'''
scored_pairwise, summary_pairwise, win_counts_pairwise = compare_runs(
    baseline_cfg,
    events=best_event_files,
    variants=variants,
    run_single_event=run_single_event,
    refine_sector=False,
    topo_kw=topo_kw,
    run_if_missing_baseline=True,
    run_if_missing_variants=True,
)
'''
def per_event_winner_abs(df_abs_scored: pd.DataFrame):
    """
    Pick the min score_abs per event_id using label indexing (safe).
    Skips events with all-NaN score_abs.
    """
    if df_abs_scored is None or df_abs_scored.empty or "score_abs" not in df_abs_scored.columns:
        return (pd.DataFrame(columns=["event_id","tag","score_abs"]),
                pd.DataFrame(columns=["tag","wins"]))

    d = df_abs_scored[np.isfinite(df_abs_scored["score_abs"])].copy()
    if d.empty:
        return (pd.DataFrame(columns=["event_id","tag","score_abs"]),
                pd.DataFrame(columns=["tag","wins"]))

    idx = d.groupby("event_id")["score_abs"].idxmin().dropna()
    winners = d.loc[idx, ["event_id","tag","score_abs"]].reset_index(drop=True)  # <-- loc, not iloc
    win_counts = (winners["tag"]
                  .value_counts()
                  .rename_axis("tag")
                  .reset_index(name="wins")
                  .sort_values("wins", ascending=False))
    return winners, win_counts


# 2) Build absolute (baseline-free) table and scores
abs_tbl = build_intrinsic_table(
    baseline_cfg,
    events=best_event_files,
    variants=variants,
    run_single_event=run_single_event,
    refine_sector=False,
    topo_kw=topo_kw,
    run_if_missing_baseline=False,   # set True if you want to autorun here too
    run_if_missing_variants=False,
)

abs_scored = add_baseline_free_scores(abs_tbl)  # you can pass custom weights=...
abs_summary = summarize_absolute_runs(abs_scored)
winners_abs, win_counts_abs = per_event_winner_abs(abs_scored)

abs_summary.to_csv(OUTPUT_DIR / "abs_summary.csv", index=False)
win_counts_abs.to_csv(OUTPUT_DIR / "win_counts.csv", index=False)

In [ ]:
from flovopy.asl.compare_runs import crawl_intrinsic_runs
abs_tbl = crawl_intrinsic_runs(OUTPUT_DIR)
abs_scored  = add_baseline_free_scores(abs_tbl)
abs_summary = summarize_absolute_runs(abs_scored)
winners_abs, win_counts_abs = per_event_winner_abs(abs_scored)

In [ ]:
# --- Run ASL per event (cell 6) ---
'''
from typing import List, Dict, Any
summaries: List[Dict[str, Any]] = []

for i, ev in zip(best_file_nums, best_event_files):
    print(f"[{i}/{len(event_files)}] {ev}")
    result = run_single_event(
        mseed_file=str(ev),
        cfg=cfg,
        refine_sector=REFINE_SECTOR,
        station_gains_df=None,
        topo_kw=topo_kw,
        debug=True,
    )
    summaries.append(result)
    break

# Summarize
df = pd.DataFrame(summaries)
display(df)

summary_csv = Path(OUTPUT_DIR) / f"{cfg.tag()}__summary.csv"
df.to_csv(summary_csv, index=False)
print(f"Summary saved to: {summary_csv}")

if not df.empty:
    n_ok = int((~df.get("error").notna()).sum()) if "error" in df.columns else len(df)
    print(f"Success: {n_ok}/{len(df)}")
'''

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np

def load_all_event_comparisons(root: Path) -> pd.DataFrame:
    """
    Crawl event folders under `root` and stack `pairwise_run_comparisons.csv`.
    Returns a tidy DF with event_id inferred from folder name.
    """
    rows = []
    for csv in root.rglob("pairwise_run_comparisons.csv"):
        try:
            df = pd.read_csv(csv)
            df["event_id"] = csv.parent.name            # the event folder name
            rows.append(df)
        except Exception as e:
            print(f"[skip] {csv}: {e}")
    if not rows:
        return pd.DataFrame()
    out = pd.concat(rows, ignore_index=True)
    # normalize label text to a short key
    out["variant"] = out["label"].astype(str)
    # guard presence of expected columns
    for c in ["mean_sep_km","delta_misfit_B_minus_A","delta_azgap_B_minus_A"]:
        if c not in out.columns: out[c] = np.nan
    return out

def add_composite_score(df: pd.DataFrame,
                        w_sep=1.0, w_misfit=0.5, w_azgap=0.1) -> pd.DataFrame:
    """
    Lower is better. Negative deltas are good if they reduce misfit/azgap.
    """
    d = df.copy()
    # z-score each metric for comparability (event-wise optional)
    # here: global z-scores; switch to per-event z if events differ strongly in scale
    for col in ["mean_sep_km","delta_misfit_B_minus_A","delta_azgap_B_minus_A"]:
        x = d[col].to_numpy(dtype=float)
        mu, sd = np.nanmean(x), np.nanstd(x) if np.nanstd(x)>0 else 1.0
        d[col+"_z"] = (x - mu)/sd
    d["score"] = (
        w_sep    * d["mean_sep_km_z"] +
        w_misfit * d["delta_misfit_B_minus_A_z"] +
        w_azgap  * d["delta_azgap_B_minus_A_z"]
    )
    return d

def summarize_variants(df: pd.DataFrame) -> pd.DataFrame:
    """
    One line per variant: mean±SE of core metrics and composite score,
    plus 'wins' (how often variant beats baseline the most for an event).
    """
    g = df.groupby("variant", dropna=False)
    agg = g.agg(
        n_events          = ("event_id", "nunique"),
        n_rows            = ("event_id", "size"),
        mean_sep_km_mean  = ("mean_sep_km", "mean"),
        mean_sep_km_med   = ("mean_sep_km", "median"),
        mean_sep_km_se    = ("mean_sep_km", lambda x: np.nanstd(x)/np.sqrt(max(1,(x.notna().sum())))),
        dmisfit_mean      = ("delta_misfit_B_minus_A", "mean"),
        dmisfit_med       = ("delta_misfit_B_minus_A", "median"),
        dazgap_mean       = ("delta_azgap_B_minus_A", "mean"),
        score_mean        = ("score", "mean"),
        score_med         = ("score", "median"),
    ).reset_index().sort_values("score_mean")
    return agg

def per_event_winner(df_scored: pd.DataFrame) -> pd.DataFrame:
    """
    For each event, pick the variant with the lowest composite score.
    """
    # keep only the best per (event_id)
    idx = df_scored.groupby("event_id")["score"].idxmin()
    winners = df_scored.loc[idx, ["event_id","variant","score"]]
    win_counts = winners.groupby("variant").size().rename("wins").reset_index()
    return winners, win_counts.sort_values("wins", ascending=False)

# --- run it ---
ROOT = OUTPUT_DIR  # your existing OUTDIR base
allcmp = load_all_event_comparisons(ROOT)
print(f"stacked rows: {len(allcmp)}, events: {allcmp['event_id'].nunique()}")

scored = add_composite_score(allcmp, w_sep=1.0, w_misfit=0.5, w_azgap=0.1)
summary = summarize_variants(scored)
winners, win_counts = per_event_winner(scored)

# quick looks
display(summary.head(10))
display(win_counts)

In [ ]:
abs_tbl    = build_intrinsic_table(baseline_cfg, events=best_event_files, variants=variants,
                                   run_single_event=run_single_event, refine_sector=False,
                                   topo_kw=topo_kw, run_if_missing_baseline=True,
                                   run_if_missing_variants=True)

abs_scored = add_baseline_free_scores(abs_tbl)  # or pass custom weights=
abs_summary = summarize_absolute_runs(abs_scored)
winners_abs, win_counts_abs = per_event_winner_abs(abs_scored)

# Run all events efficiently

In [ ]:
print(INPUT_DIR)
print(cfg)
print(topo_kw)
print(REFINE_SECTOR)
'''
run_all_events(
    input_dir=INPUT_DIR,
    station_gains_df = None,
    cfg=cfg,
    refine_sector=REFINE_SECTOR,
    topo_kw=topo_kw,
    debug=True,
    max_events=999999,
    use_multiprocessing=True,
    workers=4,
)
'''

# Run Monte Carlo sweep of parameters for 1 event


In [ ]:

from flovopy.asl.wrappers2 import run_event_monte_carlo
from flovopy.processing.sam import VSAM, DSAM
'''
# Simple 6-draw sweep (replace with your own priors/sequences)
configs = ASLConfig.generate_config_list(
    inventory=None,
    output_base=None,
    gridobj=None,
    global_cache=None,      
    wave_kinds=("surface","body"),
    station_corr_tables=(station_corrections_df), #annual_station_corrections_df),
    speeds=(1.0, 3.0),
    Qs=(23, 1000),
    dist_modes=("3d",), # 2d needs a different grid and different distance and amplitude corrections
    misfit_engines=("l2","r2", "lin"),
    peakfs=(2.0, 8.0),
    window_seconds = 5.0, # change to be a tuple 10.0) not implemented yet
    min_stations = 5,
    sam_class = (VSAM), #, DSAM), # not implemented yet
    sam_metric = ("mean"),# "median", "rms", "VT", "LP"), # this doesn't seem to be implemented yet
    # context can be set later; set here if you like:
    debug=False,
)

configs = ASLConfig.generate_config_list(
    inventory=None,
    output_base=None,
    gridobj=None,
    global_cache=None,      
    wave_kinds=("surface",),
    station_corr_tables=(station_corrections_df), #annual_station_corrections_df),
    speeds=(1.0, 3.0),
    Qs=(23, 1000),
    dist_modes=("3d",), # 2d needs a different grid and different distance and amplitude corrections
    misfit_engines=("l2"),
    peakfs=(8.0),
    window_seconds = 5.0, # change to be a tuple 10.0) not implemented yet
    min_stations = 5,
    sam_class = (VSAM), #, DSAM), # not implemented yet
    sam_metric = ("mean"),# "median", "rms", "VT", "LP"), # this doesn't seem to be implemented yet
    # context can be set later; set here if you like:
    debug=False,
)


configs = ASLConfig.generate_config_list(    
    inventory=INV,
    output_base=str(OUTPUT_DIR),
    gridobj=gridobj,
    global_cache=GLOBAL_CACHE,
) 

print(len(configs))
'''


In [ ]:

# Shared run context
mseed_file   = event_files[116]
'''
results = run_event_monte_carlo(
    mseed_file=mseed_file,
    configs=configs,
    inventory=INV,
    output_base=str(OUTPUT_DIR),
    gridobj=gridobj,
    topo_kw=topo_kw,
    station_gains_df=None,
    parallel=False,
    max_workers=1,
    global_cache=GLOBAL_CACHE,
    debug=True,
)

# Inspect or summarize results as needed
n_ok = sum(1 for r in results if "error" not in r)
print(f"[MC] Completed {n_ok}/{len(results)} runs OK")
'''